In [ ]:
import mitsuba as mi
import time

In [ ]:
mi.set_variant('cuda_ad_rgb')

In [ ]:
def render(scene_str: str, verbose = True):
    scene = mi.load_file(scene_str)
    start = time.time()
    image = mi.render(scene)
    end = time.time()
    if verbose:
        print(end-start)
    return image

# 1- Basic setup

These inital renders show the basic setup: a ceiling emitting light, an object (here, a sphere containing some homogeneous medium) and a floor below, first with an area emitter then with a directional area emitter. To show the setup, the perspective sensor is placed to the side of the setup.

In [ ]:
mi.Bitmap(render("scenes/area_setup.xml"))

Characteristics:
- whole ceiling = area emitter, path tracer
- BSDF for both ceiling and floor : diffuse with reflectance = (0.5, 0.5, 0.5)
- perspective sensor with box rfilter
- sphere (homogenous media parameters): sigma_t = 10, albedo = 0.5, phase: hg with g = 0.5

-----

In [ ]:
mi.Bitmap(render("scenes/directionalarea_setup.xml"))

Characteristics:
- whole ceiling = directionalarea emitter, ptracer integrator
- BSDF for both ceiling and floor : diffuse with reflectance = (0.5, 0.5, 0.5)
- perspective sensor with box rfilter
- sphere: sigma_t = 10, albedo = 0.5, phase: hg with g = 0.5

> Question:
> Why is there a shadow rectangle and corner on the floor?

Note that using a slightly different and "more exact" transform to_world, we do not get that shadow and the lighting is significantly different. In particular, note that the radiance for the light emitter used is the same. Thus assuming high sensibility of directionalarea to transforms, which makes sense.

Transform before:
```
<transform name="to_world">
	<matrix value="-1 7.64274e-015 -1.74846e-007 0 8.74228e-008 8.74228e-008 -2 2 0 -1 -4.37114e-008 0 0 0 0 1" />
</transform>
```
It is a numerically close equivalent to the new, simpler transform:
```
<transform name="to_world">
	<translate x="0" y="0" z="2" />
	<rotate x="1" y="0" z="0" angle="-90" />
</transform>
```

In [ ]:
mi.Bitmap(render("scenes/directionalarea_setup_2.xml"))

# 2- Faulty renders with irradiance meter

The sensor is changed. Instead of giving a side view of the scene, the sensor is now an irradiance meter embedded into the floor.

In [ ]:
mi.Bitmap(render("scenes/area_res_1.xml"))

Characteristics:
- whole ceiling = *area* emitter, path integrator (radiance (0.1, 0.1, 0.1))
- BSDF for both ceiling and floor : diffuse with reflectance = (0, 0, 0)
- irradiancemeter sensor on the floor with box rfilter
- sphere: **sigma_t = 100000**, albedo = 0.5, phase: hg with g = 0.5

> Question:
> How come we almost don't see the ball at all even with this big of an extinction coefficient sigma_t?
> NB: with more radiance emitted from the ceiling, it becomes even harder to see the ball.

-----

In [ ]:
mi.Bitmap(render("scenes/directionalarea_res_1.xml"))

Characteristics:
- whole ceiling = *directionalarea* emitter, ptracer integrator (with radiance (100000, 100000, 100000)
- BSDF for both ceiling and floor : diffuse with reflectance = (0, 0, 0)
- irradiancemeter sensor on the floor with box rfilter
- sphere: **sigma_t = 0**, albedo = 0.5, phase: hg with g = 0.5
> Question:
> Dual question: how come we cannot see any light when the ceiling should shine light directly towards the floor?
> Note: upping the reflectance of the floor and ceiling did not change anything, as seen below.

-----

In [ ]:
mi.Bitmap(render("scenes/directionalarea_res_2.xml"))

Characteristics:
- whole ceiling = *directionalarea* emitter, ptracer integrator (with radiance (100000, 100000, 100000)
- BSDF for both ceiling and floor : diffuse with reflectance = (0.5, 0.5, 0.5)
- irradiancemeter sensor on the floor with box rfilter
- sphere: **sigma_t = 0**, albedo = 0.5, phase: hg with g = 0.5

Also note that the floor is flipped in this render compared to the preivou

-----

# 3- Faulty renders as seen by perspective sensor

In [ ]:
mi.Bitmap(render("scenes/area_setup_no_scattering_1.xml"))

Characteristics:
- whole ceiling = area emitter, path tracer
- BSDF for both ceiling and floor : diffuse with reflectance = (0.5, 0.5, 0.5)
- perspective sensor with box rfilter
- sphere: **sigma_t = 0**, albedo = 0.5, phase: hg with **g = 0.99**

> Questions:
> Firstly, why is the ball visible with a 0 density sigma_t?
> Secondly, with g close to 1, the Henyey-Greenstein phase function should almost scatter forward. Why does the render not reflect that, as it is almost the same as when g = 0.5? Note that the exact same result was obtained using the following phase function which should only perform pure forward scattering:
```
<phase type="tabphase">
    <string name="values" value="0,1"/>
</phase>
```

-----

In [ ]:
mi.Bitmap(render("scenes/directionalarea_setup_no_scattering_1.xml"))

Characteristics:
- whole ceiling = directionalarea emitter, ptracer integrator
- BSDF for both ceiling and floor : diffuse with reflectance = (0.5, 0.5, 0.5)
- perspective sensor with box rfilter
- sphere: **sigma_t = 0**, albedo = 0.5, phase: hg with **g = 0.99**

> Question:
> Same as the previous question. Here again, varying the parameters sigma_t and g seems to have no effect on the render.

-----

In [ ]:
mi.Bitmap(render("scenes/directionalarea_setup_no_scattering_2.xml"))

Characteristics:
- whole ceiling = directionalarea emitter, ptracer integrator
- BSDF for both ceiling and floor : diffuse with reflectance = (0.5, 0.5, 0.5)
- perspective sensor with box rfilter
- sphere: sigma_t = 0, **albedo = 1**, phase: hg with g = 0.99

> Question:
> Same as the previous question again. albedo is not affecting the render either.

-----

# 4- Renders that look closer to what we want / expect
but that are still not on the money.

In [ ]:
mi.Bitmap(render("scenes/area_small_light_setup.xml"))

This render just to show a different setup:
- smaller *area* emitter, path integrator
- BSDF for both ceiling and floor : diffuse with reflectance = (1, 1, 1)
- BSDF for light rectangle: diffuse with reflectance = (0, 0, 0)
- perspective sensor with box rfilter
- sphere: **sigma_t = 0**, albedo = 0, g = 0.99
> Question
> Why is the ball visible with a density 0?

In [ ]:
mi.Bitmap(render("scenes/area_res_2.xml"))

Characteristics:
- smaller *area* emitter, path integrator (radiance (1, 1, 1))
- BSDF for both ceiling and floor : null
- irradiancemeter sensor on the floor with box rfilter
- sphere: **sigma_t = 0**, albedo = 0, phase: hg with g = 0.5
> Question
> Same as above.

-----